In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [10]:

main_webUrl = "https://www.rumah123.com/jual/bali/rumah/?page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [11]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [12]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2079

In [13]:
dataframe = {}
dataframe["Nama"] = []
dataframe["Harga"] = []

In [ ]:
for i in range(1, 200):
    webUrl = f"https://www.rumah123.com/jual/bali/rumah/?page={i}"
    req = requests.get(webUrl, headers=headers)
    time.sleep(3)
    
    html = BeautifulSoup(req.content, "html5lib")
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            time.sleep(2)
            
            sub_html = BeautifulSoup(sub_url.content, "html5lib")

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe.keys():
                dataframe[prop_key] = []
            
            if prop_key in dataframe.keys():
                dataframe[prop_key].append(prop_value)
            
        print("")


/properti/badung/hos15317714/

/properti/badung/hos19293379/

/properti/badung/hos19279689/

/properti/denpasar/hos19150688/

/properti/denpasar/hos19293343/

/properti/badung/hos15261445/

/properti/badung/hos19293331/

/properti/denpasar/hos19293292/

/properti/denpasar/hos19232276/

/properti/denpasar/hos19150628/

/properti/denpasar/hos19293209/

/properti/badung/hos19293154/

/properti/badung/hos18762720/

/properti/denpasar/hos19110266/

/properti/gianyar/hos19292517/

/properti/tabanan/hos16610056/

/properti/badung/hos19014483/

/properti/badung/hos19015111/

/properti/badung/hos19170593/

/properti/badung/hos19019826/

/properti/denpasar/hos19237232/

/properti/denpasar/hos19237207/

/properti/badung/hos19236945/

/properti/denpasar/hos19236951/

/properti/badung/hos18892120/

/properti/badung/hos19236906/

/properti/badung/hos16747143/

/properti/badung/hos18856015/

/properti/denpasar/hos18500003/

/properti/buleleng/hos16755346/

/properti/denpasar/hos19235786/

/properti/d

In [15]:
rumah123_df = pd.DataFrame.from_dict(dataframe, orient='index').T
rumah123_df = rumah123_df.fillna(0)
rumah123_df.to_json("Provinsi_Bali3.json", orient="records")

In [16]:
len(rumah123_df)

334